In [ ]:
!pip install pandas

In [ ]:
!pip install torch

In [ ]:
!pip install transformers


In [ ]:
import pandas as pd
from transformers import BertTokenizer, BertModel
import torch

# Load ProtBERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('Rostlab/prot_bert', do_lower_case=False)  # Ensures sequences are not lowercased
model = BertModel.from_pretrained('Rostlab/prot_bert')

# Function to get embeddings for a protein sequence
def get_protbert_embeddings(sequence):
    # Ensure the sequence is properly formatted (uppercase amino acids with no spaces)
    sequence = " ".join(list(sequence))  # Insert spaces between each amino acid
    inputs = tokenizer(sequence, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    # Mean pooling to get fixed-size embeddings for the whole sequence
    return outputs.last_hidden_state.mean(dim=1).numpy()

# Load your CSV file that contains the amino acid sequences
# Assume the column containing sequences is named 'Sequence'
data = pd.read_csv('EDD.csv')  # Replace with your actual file path

# Prepare to store embeddings
embeddings = []

# Iterate over all sequences and generate embeddings
for seq in data[' Sequence']:
    embedding = get_protbert_embeddings(seq)
    embeddings.append(embedding[0])  # Append the first (and only) embedding

# Convert the embeddings list to a DataFrame
embeddings_df = pd.DataFrame(embeddings)

# Save the embeddings to a CSV file
embeddings_df.to_csv('EDD_embedding.csv', index=False)  # Replace with desired output path

print(f"Protein embeddings saved to 'protein_embeddings.csv'.")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/81.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/361 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

Protein embeddings saved to 'protein_embeddings.csv'.


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Load dataset from a CSV file
# Replace 'your_file.csv' with the path to your dataset file
data = pd.read_csv('TGprotein_embed.csv')

# Assume the last column is the target variable, adjust accordingly
X = data.iloc[:, :-1]  # Features (all columns except the last one)
y = data.iloc[:, -1]   # Target (last column)

# Check the type of the target variable
if np.issubdtype(y.dtype, np.number):
    # If y is continuous, convert it to categorical
    num_classes = 3  # Adjust based on your needs
    bins = np.linspace(start=y.min(), stop=y.max(), num=num_classes + 1)
    y = np.digitize(y, bins) - 1  # Binning into classes (0 to num_classes-1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create the Naive Bayes model
model = GaussianNB()

# Cross-validation
cross_val_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')

# Print cross-validation results
print(f'Cross-validation Accuracy: {cross_val_scores.mean()}')

# Fit the model on the training data
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate evaluation metrics with multiclass averaging
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')  # Change to 'macro', 'micro', or 'weighted'
recall = recall_score(y_test, y_pred, average='macro')        # Change to 'macro', 'micro', or 'weighted'
f1 = f1_score(y_test, y_pred, average='macro')                # Change to 'macro', 'micro', or 'weighted'

# Print evaluation results
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Cross-validation Accuracy: 0.6648357614695504
Accuracy: 0.6594427244582043
Precision: 0.5429258015669234
Recall: 0.6983941983941984
F1 Score: 0.5631101646294622


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Load dataset from a CSV file
# Replace 'TGprotein_embed.csv' with the path to your dataset file
data = pd.read_csv('TGprotein_embed.csv')

# Assume the last column is the target variable, adjust accordingly
X = data.iloc[:, :-1]  # Features (all columns except the last one)
y = data.iloc[:, -1]   # Target (last column)

# Check if y is continuous and convert to categorical if necessary
if np.issubdtype(y.dtype, np.number):
    # Define number of classes or bins
    num_classes = 3  # Adjust as needed
    # Create bins for continuous values
    bins = np.linspace(y.min(), y.max(), num_classes + 1)
    y = np.digitize(y, bins) - 1  # Convert to discrete classes

# Ensure y is of type integer
y = y.astype(int)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create the Naive Bayes model
model = GaussianNB()

# Cross-validation
cross_val_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')

# Print cross-validation results
print(f'Cross-validation Accuracy: {cross_val_scores.mean()}')

# Fit the model on the training data
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate evaluation metrics with multiclass averaging
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')  # Change to 'macro', 'micro', or 'weighted'
recall = recall_score(y_test, y_pred, average='macro')        # Change to 'macro', 'micro', or 'weighted'
f1 = f1_score(y_test, y_pred, average='macro')                # Change to 'macro', 'micro', or 'weighted'

# Print evaluation results
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Cross-validation Accuracy: 0.6648357614695504
Accuracy: 0.6594427244582043
Precision: 0.5429258015669234
Recall: 0.6983941983941984
F1 Score: 0.5631101646294622


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Load dataset from a CSV file
# Replace 'EDD_embedding.csv' with the path to your dataset file
data = pd.read_csv('EDD_embedding.csv')

# Assume the last column is the target variable, adjust accordingly
X = data.iloc[:, :-1]  # Features (all columns except the last one)
y = data.iloc[:, -1]   # Target (last column)

# Check if y is continuous and convert to categorical if necessary
if np.issubdtype(y.dtype, np.number):
    # Define number of classes or bins
    num_classes = 3  # Adjust as needed
    # Create bins for continuous values
    bins = np.linspace(y.min(), y.max(), num_classes + 1)
    y = np.digitize(y, bins) - 1  # Convert to discrete classes

# Ensure y is of type integer
y = y.astype(int)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create the KNN model
k = 5  # You can change this value to find the optimal number of neighbors
model = KNeighborsClassifier(n_neighbors=k)

# Cross-validation
cross_val_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')

# Print cross-validation results
print(f'Cross-validation Accuracy: {cross_val_scores.mean()}')

# Fit the model on the training data
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate evaluation metrics with multiclass averaging
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')  # Change to 'macro', 'micro', or 'weighted'
recall = recall_score(y_test, y_pred, average='macro')        # Change to 'macro', 'micro', or 'weighted'
f1 = f1_score(y_test, y_pred, average='macro')                # Change to 'macro', 'micro', or 'weighted'

# Print evaluation results
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Cross-validation Accuracy: 0.7604268370264714
Accuracy: 0.7865497076023392
Precision: 0.7217478184614942
Recall: 0.678269644286767
F1 Score: 0.6952479036577076


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Load dataset from a CSV file
# Replace 'EDD_embedding.csv' with the path to your dataset file
data = pd.read_csv('TGprotein_embed.csv')

# Assume the last column is the target variable, adjust accordingly
X = data.iloc[:, :-1]  # Features (all columns except the last one)
y = data.iloc[:, -1]   # Target (last column)

# Check if y is continuous and convert to categorical if necessary
if np.issubdtype(y.dtype, np.number):
    # Define number of classes or bins
    num_classes = 3  # Adjust as needed
    # Create bins for continuous values
    bins = np.linspace(y.min(), y.max(), num_classes + 1)
    y = np.digitize(y, bins) - 1  # Convert to discrete classes

# Ensure y is of type integer
y = y.astype(int)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create the Random Forest model
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Cross-validation
cross_val_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')

# Print cross-validation results
print(f'Cross-validation Accuracy: {cross_val_scores.mean()}')

# Fit the model on the training data
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate evaluation metrics with multiclass averaging
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')  # Change to 'macro', 'micro', or 'weighted'
recall = recall_score(y_test, y_pred, average='macro')        # Change to 'macro', 'micro', or 'weighted'
f1 = f1_score(y_test, y_pred, average='macro')                # Change to 'macro', 'micro', or 'weighted'

# Print evaluation results
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Cross-validation Accuracy: 0.8433022652550297
Accuracy: 0.8390092879256966
Precision: 0.7553138902619873
Recall: 0.5325340676217869
F1 Score: 0.5838057101253314


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Load dataset from a CSV file
data = pd.read_csv('TGprotein_embed.csv')

# Assume the last column is the target variable, adjust accordingly
X = data.iloc[:, :-1]  # Features (all columns except the last one)
y = data.iloc[:, -1]   # Target (last column)

# Check if y is continuous and convert to categorical if necessary
if np.issubdtype(y.dtype, np.number):
    # Define number of classes or bins
    num_classes = 3  # Adjust as needed
    # Create bins for continuous values
    bins = np.linspace(y.min(), y.max(), num_classes + 1)
    y = np.digitize(y, bins) - 1  # Convert to discrete classes

# Ensure y is of type integer
y = y.astype(int)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create the Logistic Regression model
model = LogisticRegression(max_iter=300, multi_class='multinomial', solver='lbfgs')

# Cross-validation
cross_val_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')

# Print cross-validation results
print(f'Cross-validation Accuracy: {cross_val_scores.mean()}')

# Fit the model on the training data
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate evaluation metrics with multiclass averaging
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')  # Change to 'macro', 'micro', or 'weighted'
recall = recall_score(y_test, y_pred, average='macro')        # Change to 'macro', 'micro', or 'weighted'
f1 = f1_score(y_test, y_pred, average='macro')                # Change to 'macro', 'micro', or 'weighted'

# Print evaluation results
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default 

Cross-validation Accuracy: 0.8556993333936598


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Accuracy: 0.8637770897832817
Precision: 0.8091288116670157
Recall: 0.5998741349618543
F1 Score: 0.6590909090909091


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Load dataset from a CSV file
data = pd.read_csv('TGprotein_embed.csv')

# Assume the last column is the target variable, adjust accordingly
X = data.iloc[:, :-1]  # Features (all columns except the last one)
y = data.iloc[:, -1]   # Target (last column)

# Check if y is continuous and convert to categorical if necessary
if np.issubdtype(y.dtype, np.number):
    # Define number of classes or bins
    num_classes = 3  # Adjust as needed
    # Create bins for continuous values
    bins = np.linspace(y.min(), y.max(), num_classes + 1)
    y = np.digitize(y, bins) - 1  # Convert to discrete classes

# Ensure y is of type integer
y = y.astype(int)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create the SVM model
model = SVC(kernel='linear')  # You can change the kernel to 'rbf', 'poly', etc.

# Cross-validation
cross_val_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')

# Print cross-validation results
print(f'Cross-validation Accuracy: {cross_val_scores.mean()}')

# Fit the model on the training data
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate evaluation metrics with multiclass averaging
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')  # Change to 'macro', 'micro', or 'weighted'
recall = recall_score(y_test, y_pred, average='macro')        # Change to 'macro', 'micro', or 'weighted'
f1 = f1_score(y_test, y_pred, average='macro')                # Change to 'macro', 'micro', or 'weighted'

# Print evaluation results
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Cross-validation Accuracy: 0.8525955418815794
Accuracy: 0.8637770897832817
Precision: 0.7616173049110232
Recall: 0.5867400955120253
F1 Score: 0.6305016964630124


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Load dataset from a CSV file
data = pd.read_csv('TGprotein_embed.csv')

# Assume the last column is the target variable, adjust accordingly
X = data.iloc[:, :-1]  # Features (all columns except the last one)
y = data.iloc[:, -1]   # Target (last column)

# Check if y is continuous and convert to categorical if necessary
if np.issubdtype(y.dtype, np.number):
    # Define number of classes or bins
    num_classes = 3  # Adjust as needed
    # Create bins for continuous values
    bins = np.linspace(y.min(), y.max(), num_classes + 1)
    y = np.digitize(y, bins) - 1  # Convert to discrete classes

# Ensure y is of type integer
y = y.astype(int)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create the MLP model
model = MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, random_state=42)

# Cross-validation
cross_val_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')

# Print cross-validation results
print(f'Cross-validation Accuracy: {cross_val_scores.mean()}')

# Fit the model on the training data
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate evaluation metrics with multiclass averaging
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')  # Change to 'macro', 'micro', or 'weighted'
recall = recall_score(y_test, y_pred, average='macro')        # Change to 'macro', 'micro', or 'weighted'
f1 = f1_score(y_test, y_pred, average='macro')                # Change to 'macro', 'micro', or 'weighted'

# Print evaluation results
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochast

Cross-validation Accuracy: 0.8564684945555454
Accuracy: 0.8668730650154799
Precision: 0.743977315335583
Recall: 0.7136434066258627
F1 Score: 0.7277365014612817


In [ ]:
!pip install scikeras[tensorflow]

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from scikeras.wrappers import KerasClassifier
from tensorflow import keras
from tensorflow.keras import layers

# Load dataset from a CSV file
data = pd.read_csv('TGprotein_embed.csv')

# Assume the last column is the target variable
X = data.iloc[:, :-1].values  # Features (all columns except the last one)
y = data.iloc[:, -1].values    # Target (last column)

# Check if y is continuous and convert to categorical if necessary
if np.issubdtype(y.dtype, np.number):
    num_classes = len(np.unique(y))  # Determine the number of unique classes
    bins = np.linspace(y.min(), y.max(), num_classes + 1)
    y = np.digitize(y, bins) - 1  # Convert to discrete classes

# Ensure y is of type integer
y = y.astype(int)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Function to create the ANN model
def create_model():
    model = keras.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(len(np.unique(y)), activation='softmax'))  # Output layer for multiclass classification
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Create KerasClassifier
model = KerasClassifier(model=create_model, epochs=100, batch_size=10, verbose=0)

# Cross-validation
cross_val_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')

# Print cross-validation results
print(f'Cross-validation Accuracy: {cross_val_scores.mean()}')

# Fit the model on the training data
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

# Print evaluation results
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

# Additional evaluation
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a laye

Cross-validation Accuracy: 0.002325581395348837


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load dataset from a CSV file
data = pd.read_csv('TGprotein_embed.csv')

# Assuming the last column is the target variable and the rest are features
X = data.iloc[:, :-1].values  # Features (all columns except the last one)
y = data.iloc[:, -1].values    # Target (last column)

# Adjust the class labels to start from 0 if they start from 1
if y.min() == 1:  # Check if the labels start from 1
    y = y - 1  # Adjust class labels to start from 0

# Print unique classes for debugging
print("Unique classes in target variable:", np.unique(y))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the XGBoost model
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

# Cross-validation
cross_val_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')

# Print cross-validation results
print(f'Cross-validation Accuracy: {cross_val_scores.mean()}')

# Fit the model on the training data
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate evaluation metrics with multiclass averaging
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')  # Use 'macro' for multiclass
recall = recall_score(y_test, y_pred, average='macro')        # Use 'macro' for multiclass
f1 = f1_score(y_test, y_pred, average='macro')                # Use 'macro' for multiclass

# Print evaluation results
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')


Unique classes in target variable: [-0.12439344 -0.09415875 -0.08807844 ...  0.07992235  0.09188595
  0.10036241]


ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/xgboost/core.py", line 726, in inner_f
    return func(**kwargs)
  File "/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py", line 1491, in fit
    raise ValueError(
ValueError: Invalid classes inferred from unique values of `y`.  Expected: [   0    1    2 ... 1028 1029 1030], got [-0.12439344 -0.09415875 -0.08807844 ...  0.0752719   0.07992235
  0.10036241]

----------------------------------------------------------------

In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load dataset from a CSV file
data = pd.read_csv('TGprotein_embed.csv')

# Assuming the last column is the target variable and the rest are features
X = data.iloc[:, :-1].values  # Features (all columns except the last one)
y = data.iloc[:, -1].values    # Target (last column)

# Check if the target variable is continuous
if np.issubdtype(y.dtype, np.number):
    # Binning continuous values into discrete classes
    num_bins = 3  # Adjust based on your needs
    y = pd.cut(y, bins=num_bins, labels=False)  # Creates 0, 1, 2 classes

# Print unique classes for debugging
print("Unique classes in target variable:", np.unique(y))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the XGBoost model
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

# Cross-validation
cross_val_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')

# Print cross-validation results
print(f'Cross-validation Accuracy: {cross_val_scores.mean()}')

# Fit the model on the training data
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

# Print evaluation results
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')


Unique classes in target variable: [0 1 2]


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [04:41:24] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [04:41:48] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [04:42:08] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [04:42:26] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [04:42:44] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_e

Cross-validation Accuracy: 0.8487256055258952


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [04:43:04] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy: 0.8637770897832817
Precision: 0.8099866548042703
Recall: 0.6088528106071965
F1 Score: 0.6708656330749353
